<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Validate_Classifier_Timing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from glob import glob
import pickle
import numpy as np
import torch
import torch.nn as nn

# ─── Konstanten ───────────────────────────────────────────────────────────────
MAX_LENGTH = 400
SCALER_STYLE_FILE = "scaler_schwung.pkl"
ENCODER_STYLE_FILE = "encoder_schwung.pkl"
MODEL_STYLE_FILE = "ski_schwung_classifier.pt"
MODEL_TURN_FILE = "left_right_classifier.pt"
all_features = "*PS_features.pkl"

# ─── Scaler & Encoder für Fahrstil laden ───────────────────────────────────────
with open(SCALER_STYLE_FILE, "rb") as f:
    scaler_style = pickle.load(f)
with open(ENCODER_STYLE_FILE, "rb") as f:
    le_style = pickle.load(f)

# ─── Architektur-Komponenten ───────────────────────────────────────────────────
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size * 4, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, outputs):
        hidden = hidden.unsqueeze(1).repeat(1, outputs.size(1), 1)
        combined = torch.cat((hidden, outputs), dim=2)
        energy = torch.tanh(self.attn(combined))
        attention = torch.softmax(self.v(energy).squeeze(2), dim=1)
        return torch.sum(attention.unsqueeze(2) * outputs, dim=1)

class SkiSwingLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.4
        )
        self.attention = Attention(hidden_size)
        self.dropout = nn.Dropout(0.6)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        outputs, (hidden, _) = self.lstm(x)
        hidden_combined = torch.cat((hidden[-2], hidden[-1]), dim=1)
        context = self.attention(hidden_combined, outputs)
        return self.fc(self.dropout(context))

class FrameWiseLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.3
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(self.dropout(out))

# ─── Modelle laden ─────────────────────────────────────────────────────────────
torch.serialization.add_safe_globals([SkiSwingLSTM, Attention, FrameWiseLSTM])
model_style = torch.load(MODEL_STYLE_FILE, map_location="cpu")
model_style.eval()
model_turn = torch.load(MODEL_TURN_FILE, map_location="cpu", weights_only=False)
model_turn.eval()

# ─── Metriken initialisieren ───────────────────────────────────────────────────
gesamtabweichung = 0
gesamtschwünge = 0

# ─── Processing Loop ───────────────────────────────────────────────────────────
for file in glob(all_features):
    # Features laden
    with open(file, "rb") as f:
        feats = pickle.load(f)

    # 1) Fahrstil vorhersagen
    frames = sorted(k for k in feats["COM_to_ground"] if isinstance(k, int))
    style_data = []
    for frame in frames:
        row = [
            feats["COM_to_ground"][frame],
            feats["knee_angles_right"][frame],
            feats["knee_angles_left"][frame],
        ]
        for axis in feats["joint_angles"][frame]:
            row += list(feats["joint_angles"][frame][axis].values())
        for axis in feats["axis_angles"][frame]:
            row += list(feats["axis_angles"][frame][axis].values())
        row += list(feats["COM_angles"][frame].values())
        style_data.append(row)

    style_arr = np.array(style_data, dtype=np.float32)
    if style_arr.shape[0] < MAX_LENGTH:
        pad = np.zeros((MAX_LENGTH - style_arr.shape[0], style_arr.shape[1]), dtype=np.float32)
        style_arr = np.vstack([style_arr, pad])
    else:
        style_arr = style_arr[:MAX_LENGTH]

    style_scaled = scaler_style.transform(style_arr)
    x_style = torch.tensor(style_scaled, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        logits_style = model_style(x_style)
        style_idx = int(logits_style.argmax(dim=1).item())
    style_str = le_style.inverse_transform([style_idx])[0]
    print("Vorhergesagte Schwungart:", style_str)

    # 2) Turn-Klassifikation
    seq_turn = []
    for frame in frames:
        v = [
            feats["COM_to_ground"][frame],
            feats["knee_angles_right"][frame],
            feats["knee_angles_left"][frame],
        ]
        for axis in feats["joint_angles"][frame]:
            v += list(feats["joint_angles"][frame][axis].values())
        for axis in feats["axis_angles"][frame]:
            v += list(feats["axis_angles"][frame][axis].values())
        v += list(feats["COM_angles"][frame].values())
        v.append(style_idx)
        seq_turn.append(v)

    if not seq_turn:
        print(f"{file}: Keine Frames für Turn-Seq, überspringe.")
        continue

    arr_turn = np.array(seq_turn, dtype=np.float32)
    x_turn = torch.tensor(arr_turn, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        logits_turn = model_turn(x_turn)
        preds = logits_turn.argmax(dim=-1).squeeze().cpu().numpy()

    labels = ["Linksschwung" if p == 1 else "Rechtsschwung" for p in preds]
    print("Schwung pro Frame:", labels)

    # 3) Abweichungs-Berechnung
    abweichung = 0
    schwünge = 0
    prev_val = 0
    for idx, real_val in feats.get("schwung_labels", {}).items():
        if prev_val == real_val:
            continue
        schwünge += 1
        if real_val == preds[idx]:
            counter = idx
            while counter > 0 and preds[counter] == real_val:
                abweichung += 1
                counter -= 1
        else:
            counter = idx
            while counter < len(preds) and preds[counter] != real_val:
                abweichung += 1
                counter += 1
        prev_val = real_val

    print(abweichung)
    print(schwünge)
    print("Durchschnittliche Abweichung in Frames:", abweichung/schwünge)
    print("Durchschnittliche Abweichung in Sekunden:", (abweichung/schwünge)/25)

    gesamtabweichung += abweichung
    gesamtschwünge += schwünge

# 4) Gesamtergebnis
if gesamtschwünge > 0:
    avg_frames = gesamtabweichung / gesamtschwünge
    avg_secs = avg_frames / 25
    print("Durchschnittliche Gesamtabweichung in Frames:", avg_frames)
    print("Durchschnittliche Gesamtabweichung in Sekunden:", avg_secs)
else:
    print("Keine Schwünge gefunden.")


Vorhergesagte Schwungart: PS
Schwung pro Frame: ['Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtss

In [8]:
# Hier der Code, um nur ein einziges File anzuschauen

abweichung = 0
schwünge = 0
prevValue = 0

for index, real_val in features["schwung_labels"].items():
  if (prevValue == real_val):
    continue
  else:
    schwünge += 1
    if (real_val == preds[index]):
      # wenn der echte Wert der prediction entspricht, dann war Schwungwechsel entweder gleich oder zuvor von Modell erkannt
      counter = index
      while counter > 0:
        if (preds[counter] != real_val):
          # wenn der vorherige Wert sich vom momentanten unterscheidet, dann hat das Modell den Schwungwechsel auch genau auf diesen Frame vorhergesagt (Modell & Realität gleich)
          break
        else:
          abweichung += 1
          counter -= 1
    else:
      # in diesem Fall entspricht der reale Wert nicht dem Wert der Prediction, d.h. die Prediction ändert ihren Wert erst zu einem späteren Wert
      counter = index
      while counter <= len(preds):
        if (preds[counter] == real_val):
          # sobald die prediction mit dem realen Wert übereinstimmt, bricht man aus der Schleife
          break
        else:
          # solange der prediction Wert nicht mit dem realen übereinstimmt, wird die Abweichung erhöht und der counter ebenfalls
          abweichung += 1
          counter += 1
  prevValue = real_val

print(abweichung)
print(schwünge)
print("Durchschnittliche Abweichung in Frames: ", abweichung/schwünge)
print("Durchschnittliche Abweichung in Sekunden: ", (abweichung/schwünge)/25)

13
4
Durchschnittliche Abweichung in Frames:  3.25
Durchschnittliche Abweichung in Sekunden:  0.13
